## 아마존 서비스 - S3 설정하기

In [ ]:
pip install awscli boto3 requests

In [2]:
# **requests**는 Python에서 HTTP 요청을 쉽게 처리하기 위한 가장 널리 사용되는 라이브러리 중 하나
# •	HTTP 요청 메서드 지원: GET, POST, PUT, DELETE, PATCH, HEAD, OPTIONS 등.
# •	JSON 데이터 처리 및 자동 디코딩.
# •	세션 유지, 쿠키 관리.
# •	인증, 프록시, 리다이렉션 처리.
# •	SSL 인증서 검증 지원.
# **urllib.parse**는 URL을 다루는 다양한 작업을 지원. URL을 파싱(분석), 구성, 조작, 그리고 인코딩/디코딩하는 데 유용함
#  **boto3**: AWS 서비스를 제어하기 위한 Python SDK
# **pathlib**: 파일 경로를 다루기 위한 모듈

import os, boto3, dotenv, urllib.parse, requests
from pathlib import Path

dotenv.load_dotenv('../.env')
AWS_ACCESS_KEY = os.getenv("AWS-ACCESS-KEY")
AWS_SECRET_KEY = os.getenv("AWS-SECRET-KEY")
S3_REGION = os.getenv("S3-BUCKET-REGION")
S3_BUCKET_NAME = os.getenv("S3-BUCKET-NAME")

In [5]:
# •	Access Key: 요청의 출처를 식별. (누가 요청했는가?)
# •	Secret Access Key: 요청의 인증을 보장. (해당 요청이 신뢰할 수 있는가?)
s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=S3_REGION,
    )

In [6]:
path = Path.cwd()
file_name = "example.jpg"
file_path = path /"../image"/file_name

### 업로드

In [ ]:
# 	URL에서 허용되지 않는 문자 처리: 파일 이름에 공백, 특수 문자(!, #, ?, /, & 등)가 포함된 경우, 이들은 URL에서 특별한 의미를 가지므로 인코딩이 필요.
# 	다국어 지원: 파일 이름에 유니코드(한글, 일본어 등)가 포함된 경우, URL에서 처리 가능한 형태로 변환해야 함.
encoded_file_name = urllib.parse.quote(file_name)
s3_client.upload_file(
            file_path, 
            S3_BUCKET_NAME, 
            file_name,
            # ExtraArgs={"ACL": "public-read"}
        )

In [9]:
# S3 URL 생성
file_url = f"https://{S3_BUCKET_NAME}.s3.{S3_REGION}.amazonaws.com/{encoded_file_name}"

In [ ]:
file_url

### 다운로드

In [ ]:
# 파일 다운로드
file_name = 'example.jpg'  # S3에 저장된 파일 이름
download_path = '../image/downloaded_example.jpg'  # 로컬에 저장할 파일 이름

try:
    s3_client.download_file(S3_BUCKET_NAME, file_name, download_path)
    print(f"File downloaded successfully to {download_path}")
except Exception as e:
    print(f"Error occurred while downloading file: {e}")

In [ ]:
response = requests.get(file_url)
if response.status_code == 200:
    with open("../image/http_example.jpg", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully")
else:
    print(f"Failed to download file. HTTP Status Code: {response.status_code}")

### 파일 삭제

In [ ]:
try:
    s3_client.delete_object(Bucket = S3_BUCKET_NAME, Key = file_name)
    print(f"File '{file_name}' successfully deleted from bucket '{S3_BUCKET_NAME}'.")
except Exception as e:
    print(f"Error occurred while deleting file: {e}")